<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/1%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D0%B8%20%D0%BE%D1%82%D1%87%D1%91%D1%82%20%D0%BF%D0%BE%20%D1%81%D0%BF%D0%B5%D1%86%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F%D0%BC%20HILTI.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HİLTİ Speklerini incelemek için oluşturuldu. Spek excellerini bir dosyada topla program sizden dosyadan excelleri seçmenizi isteyecek sonra her bir speki tek tek bir structured halde excellere kaydecek ayrıca overall bir df oluşturacak sonra bu df i excele çevirip ekleyecek.  hatalar olduğu zaman program içinde görünecektir ayrıca rapor için bir excel dosyası oluşturup bu hataları görmeni sağlayacak. Raporda oluşturulan ve hata alınan specler ayrıca hatalar  gözükecek /
Программа была создана для анализа спецификаций HILTI. Соберите Excel-файлы со спецификациями в одной папке — программа попросит вас выбрать эти файлы. Затем она поочерёдно сохранит каждую спецификацию в структурированном виде в Excel, а также сформирует общий DataFrame, который также будет преобразован в Excel и добавлен.
Ошибки, возникшие в процессе, будут отображаться внутри программы. Кроме того, будет создан отдельный Excel-файл для отчёта, в котором вы сможете увидеть эти ошибки. В отчёте будут отображены как успешно обработанные спецификации, так и те, в которых возникли ошибки, а также сами ошибки.

In [ ]:
from google.colab import files
import pandas as pd
import os
import re

In [ ]:
# Excel dosyalarını yükleyin
uploaded = files.upload()

Saving _HM-30061.xlsx to _HM-30061 (1).xlsx
Saving _HM-30062.xlsx to _HM-30062 (1).xlsx
Saving _HM-30065.xlsx to _HM-30065 (1).xlsx
Saving _HM-30066 от 04.10.2024.xlsx to _HM-30066 от 04.10.2024 (1).xlsx
Saving _HM-30069 измен 03.12.2024.xlsx to _HM-30069 измен 03.12.2024 (1).xlsx
Saving _HM-30070 12.11.2024.xlsx to _HM-30070 12.11.2024 (1).xlsx
Saving _HM-30071 от 04.12.2024.xlsx to _HM-30071 от 04.12.2024 (1).xlsx
Saving _HM-30072.xlsx to _HM-30072 (1).xlsx
Saving _HM-30073 05.12.2024.xlsx to _HM-30073 05.12.2024 (1).xlsx
Saving _HM-30074.xlsx to _HM-30074 (1).xlsx
Saving _HM-30077.xlsx to _HM-30077 (1).xlsx
Saving _HM-30078 30.01.2025.xlsx to _HM-30078 30.01.2025 (1).xlsx
Saving _HM-30079 от 06.02.2025.xlsx to _HM-30079 от 06.02.2025 (1).xlsx
Saving _HM-30080 от 17.02.2025.xlsx to _HM-30080 от 17.02.2025 (1).xlsx
Saving _HM-30081 от 13.02.2025.xlsx to _HM-30081 от 13.02.2025 (1).xlsx
Saving _HM-30082 от 19.03.2025.xlsx to _HM-30082 от 19.03.2025 (1).xlsx
Saving _HM-30083 от 19.03.20

In [ ]:
# Tüm temiz veriler burada birikecek
master_df = pd.DataFrame()

In [ ]:
# Durum raporu için tablo
rapor_kayitlari = []

In [ ]:
# Her dosyayı sırayla işle
for filename in uploaded.keys():
    print(f"\n🔄 İşleniyor: {filename}")

    try:
        xls = pd.ExcelFile(filename)

        # Başlangıç verisi ilk sayfadan
        df_start = pd.read_excel(xls, sheet_name=0, header=None, dtype=str)
        start_idx = df_start[df_start.apply(lambda row: row.astype(str).str.contains(r"Спецификация\s+№", regex=True).any(), axis=1)].index

        if len(start_idx) == 0:
            mesaj = "Спецификация № bulunamadı/не найдено ❌"
            print(mesaj)
            rapor_kayitlari.append({
                "Dosya": filename,
                "Спецификация №": "Yok",
                "Tarih": "Yok",
                "Durum": mesaj
            })
            continue

        # Spesifikasyon bilgisi
        spec_row = df_start.iloc[start_idx[0]].astype(str).tolist()
        spec_text = ' '.join(spec_row)

        match_spec = re.search(r'Спецификация №\s+(\S+)', spec_text)
        spec_number = match_spec.group(1) if match_spec else 'Bilinmiyor/Неизвестный'

        match_date = re.search(r'от\s+(\d{2}\.\d{2}\.\d{4})', spec_text)
        spec_date = match_date.group(1) if match_date else 'Bilinmiyor/Неизвестный'

        # "Цена без НДС" araması tüm sayfalarda
        end_found = False
        for sheet in xls.sheet_names:
            df_check = pd.read_excel(xls, sheet_name=sheet, header=None, dtype=str)
            end_idx = df_check[df_check.apply(lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index
            if len(end_idx) > 0:
                end_sheet = sheet
                end_index = end_idx[0]
                end_found = True
                break

        if not end_found:
            mesaj = "'Цена без НДС' bulunamadı/не найдено ❌"
            print(mesaj)
            rapor_kayitlari.append({
                "Dosya/Файл": filename,
                "Спецификация №": spec_number,
                "Tarih/дата": spec_date,
                "Durum/Ситуация": mesaj
            })
            continue

        # Structured tabloyu oluştur
        if end_sheet == xls.sheet_names[0]:
            df_table = df_start.iloc[start_idx[0]+1:end_index].copy()
        else:
            df1 = df_start.iloc[start_idx[0]+1:].copy()
            df2 = pd.read_excel(xls, sheet_name=end_sheet, header=None, dtype=str).iloc[:end_index].copy()
            df_table = pd.concat([df1, df2], ignore_index=True)

        # Temizle
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Bilgileri ekle
        df_table["номер спецификации"] = spec_number
        df_table["дата"] = spec_date

        # Kaydet
        out_filename = f"cleaned_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"✅ {out_filename} oluşturuldu./был создан.")

        # Ana tabloya ekle
        master_df = pd.concat([master_df, df_table], axis=0, ignore_index=True, sort=False)

        # Rapor tablosuna kayıt
        rapor_kayitlari.append({
            "Dosya/Файл": filename,
            "Спецификация №": spec_number,
            "Tarih/дата": spec_date,
            "Durum/Ситуация": "Oluşturuldu/был создан. ✅"
        })

    except Exception as e:
        hata_mesaji = f"Hata/Ошибка: {str(e)}"
        print(f"⚠️ {hata_mesaji}")
        rapor_kayitlari.append({
            "Dosya/Файл": filename,
            "Спецификация №": "Bilinmiyor/не найдено",
            "Tarih/дата": "Bilinmiyor/не найдено",
            "Durum/Ситуация": hata_mesaji
        })


🔄 İşleniyor: _HM-30061 (1).xlsx
✅ cleaned__HM-30061 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30062 (1).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned__HM-30062 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30065 (1).xlsx
✅ cleaned__HM-30065 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30066 от 04.10.2024 (1).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned__HM-30066 от 04.10.2024 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30069 измен 03.12.2024 (1).xlsx
✅ cleaned__HM-30069 измен 03.12.2024 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30070 12.11.2024 (1).xlsx


...

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-230035 (1).xlsx
⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-230036.v1 (1).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-20230004_v3 (1).xlsx
⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: ъ_old_rev_HMС-30082 от 21.04.2025 (1).xlsx
✅ cleaned_ъ_old_rev_HMС-30082 от 21.04.2025 (1).xlsx oluşturuldu./был создан.


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
# Her dosyayı sırayla işle
for filename in uploaded.keys():
    print(f"\n🔄 İşleniyor: {filename}")

    try:
        xls = pd.ExcelFile(filename)

        # Başlangıç verisi ilk sayfadan
        df_start = pd.read_excel(xls, sheet_name=0, header=None, dtype=str)
        start_idx = df_start[df_start.apply(
            lambda row: row.astype(str).str.contains(r"Спецификация\s+№", regex=True).any(), axis=1)].index

        if len(start_idx) == 0:
            mesaj = "Спецификация № bulunamadı/не найдено ❌"
            print(mesaj)
            rapor_kayitlari.append({
                "Dosya": filename,
                "Спецификация №": "Yok",
                "Tarih": "Yok",
                "Durum": mesaj
            })
            continue

        # Spesifikasyon bilgisi
        spec_row = df_start.iloc[start_idx[0]].astype(str).tolist()
        spec_text = ' '.join(spec_row)

        match_spec = re.search(r'Спецификация\s+№\s*(\S+)', spec_text)
        spec_number = match_spec.group(1) if match_spec else 'Bilinmiyor/Неизвестный'

        match_date = re.search(r'от\s+(\d{2}\.\d{2}\.\d{4})', spec_text)
        spec_date = match_date.group(1) if match_date else 'Bilinmiyor/Неизвестный'

        # "Цена без НДС" araması tüm sayfalarda
        end_found = False
        for sheet in xls.sheet_names:
            df_check = pd.read_excel(xls, sheet_name=sheet, header=None, dtype=str)
            end_idx = df_check[df_check.apply(
                lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index
            if len(end_idx) > 0:
                end_sheet = sheet
                end_index = end_idx[0]
                end_found = True
                break

        if not end_found:
            mesaj = "'Цена без НДС' bulunamadı/не найдено ❌"
            print(mesaj)
            rapor_kayitlari.append({
                "Dosya/Файл": filename,
                "Спецификация №": spec_number,
                "Tarih/дата": spec_date,
                "Durum/Ситуация": mesaj
            })
            continue

        # Structured tabloyu oluştur
        if end_sheet == xls.sheet_names[0]:
            df_table = df_start.iloc[start_idx[0]+1:end_index].copy()
        else:
            df1 = df_start.iloc[start_idx[0]+1:].copy()
            df2 = pd.read_excel(xls, sheet_name=end_sheet, header=None, dtype=str).iloc[:end_index].copy()
            df_table = pd.concat([df1, df2], ignore_index=True)

        # Temizle
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Bilgileri ekle
        df_table["номер спецификации"] = spec_number
        df_table["дата"] = spec_date

        # Kaydet
        out_filename = f"cleaned_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"✅ {out_filename} oluşturuldu./был создан.")

        # Ana tabloya ekle
        master_df = pd.concat([master_df, df_table], axis=0, ignore_index=True, sort=False)

        # Rapor tablosuna kayıt
        rapor_kayitlari.append({
            "Dosya/Файл": filename,
            "Спецификация №": spec_number,
            "Tarih/дата": spec_date,
            "Durum/Ситуация": "Oluşturuldu/был создан. ✅"
        })

    except Exception as e:
        hata_mesaji = f"Hata/Ошибка: {str(e)}"
        print(f"⚠️ {hata_mesaji}")
        rapor_kayitlari.append({
            "Dosya/Файл": filename,
            "Спецификация №": "Bilinmiyor/не найдено",
            "Tarih/дата": "Bilinmiyor/не найдено",
            "Durum/Ситуация": hata_mesaji
        })


🔄 İşleniyor: _HM-30061 (1).xlsx
✅ cleaned__HM-30061 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30062 (1).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned__HM-30062 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30065 (1).xlsx
✅ cleaned__HM-30065 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30066 от 04.10.2024 (1).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned__HM-30066 от 04.10.2024 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30069 измен 03.12.2024 (1).xlsx
✅ cleaned__HM-30069 измен 03.12.2024 (1).xlsx oluşturuldu./был создан.

🔄 İşleniyor: _HM-30070 12.11.2024 (1).xlsx


...

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-230034 (1).xlsx
⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-230035 (1).xlsx
⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-230036.v1 (1).xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: Спецификация-HCN-20230004_v3 (1).xlsx
⚠️ Hata/Ошибка: single positional indexer is out-of-bounds

🔄 İşleniyor: ъ_old_rev_HMС-30082 от 21.04.2025 (1).xlsx
✅ cleaned_ъ_old_rev_HMС-30082 от 21.04.2025 (1).xlsx oluşturuldu./был создан.


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
# Tüm verileri kaydet
master_out = "merged_all_cleaned.xlsx"
master_df.to_excel(master_out, index=False)
print(f"\n📁 Tüm veriler {master_out} dosyasında toplandı.")
files.download(master_out)


📁 Tüm veriler merged_all_cleaned.xlsx dosyasında toplandı.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Rapor dosyasını oluştur
rapor_df = pd.DataFrame(rapor_kayitlari)
rapor_out = "rapor_durum.xlsx"
rapor_df.to_excel(rapor_out, index=False)
print(f"📋 Rapor {rapor_out} dosyasına kaydedildi.")
files.download(rapor_out)

📋 Rapor rapor_durum.xlsx dosyasına kaydedildi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for file in os.listdir():
    if file.startswith("structured_") and file.endswith(".xlsx"):
        print(f"İndiriliyor/Загрузка: {file}")
        files.download(file)